In [2]:
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import AIMessage, HumanMessage,SystemMessage
llm2str = ChatOllama(model="qwen2:7b", temperature=0)
from langchain_ollama import OllamaEmbeddings

embedding = OllamaEmbeddings(model="nomic-embed-text:v1.5")
print(embedding.embed_query("Hello world"))

[-0.137298583984375, -0.49993896484375, -3.41259765625, 0.364166259765625, -0.29769134521484375, 1.70263671875, 0.37176513671875, -1.1875, -0.59844970703125, -0.88525390625, 0.24542236328125, 1.208984375, 0.190277099609375, 1.9697265625, 0.82568359375, -1.05224609375, 0.6501235961914062, -0.612548828125, -0.9190673828125, 0.73223876953125, 0.149261474609375, -1.632080078125, -0.54248046875, 0.8173828125, 1.9300537109375, -0.04766082763671875, -0.301055908203125, 1.5107421875, 0.1049652099609375, -0.160064697265625, -0.1656036376953125, -0.248809814453125, 0.22821044921875, 0.71917724609375, 0.82666015625, 0.370849609375, 0.81683349609375, 0.3165550231933594, 0.7071533203125, 0.0792999267578125, 0.15337657928466797, -0.939697265625, 0.0965576171875, 0.36815643310546875, 1.1243896484375, 0.290863037109375, 0.29449462890625, 0.133087158203125, 1.835693359375, -0.8916015625, -0.5359115600585938, -0.124267578125, -0.2336273193359375, 1.389892578125, 2.03955078125, 0.5783615112304688, 0.9455

In [32]:
# -*- coding: utf-8 -*-

from flask import Flask
from flask import request
from flask import jsonify
# coding=UTF-8
 


# res = client.chat.completions.create(model=qwen14b_modelId, **request.get_json())



# 在这里实现你的策略
def digest_mrc(messages):
    """
    # Request:
    {
    "messages": [
        {"role": "user", "content": "今天上海的天气怎么样"},
        {"role": "system", "content": "今天上海 18 度，晴天"},
        {"role": "user", "content": "明天呢？"},
        {"role": "system", "content": "明天预计 19 度，阴天"},
        {"role": "user", "content": "广州什么天气？"}
    ]
    }

    # Response:
    {
    "rewritten": true/false
    "rewrite_query": "广州明天什么天气？"
    }
    # 如果需要改写就是true，然后query就是改写后的问题，否则就是false，query字段可有可无
    """
    rewritten = False
    rewrite_query = messages[-1]["content"]
    # 策略 1 直接语言模型 提示改写
    ## 先将message处理成合适的prompt
    prompt_list = ["<|im_start|>system\n你的任务是在多轮对话中，生成一个改写后的用户问题，以补全上下文中缺失的信息，一般的技巧是指代消岐和信息补全。<|im_end|>"]
    query = messages[-1]["content"]
    messages.pop(-1)
    for message in messages:
        if message["role"] == "user":
            prompt_list.append("<|im_start|>user\n" + message["content"] + "\n<|im_end|>")
        elif message["role"] == "system":
            prompt_list.append("<|im_start|>assistant\n" + message["content"] + "\n<|im_end|>")
        else:
            raise ValueError("Invalid role: " + message["role"])
    prompt_list.append("-----请改写下面的用户问题，以补全上下文中缺失的信息，一般的技巧是指代消岐和信息补全-----\n<|im_start|>user\n" + query + "<|im_end|>")
    prompt = "\n".join(prompt_list)
    print(prompt)
    # print(prompt)
    # res = client.chat.completions.create(model=qwen14b_modelId, prompt=prompt, temperature=0.0, max_tokens=100)
    # if res.choices[0].message.content.strip() != query:
    #     rewritten = True
    #     rewrite_query = res.choices[0].message.content.strip()
    rewrite_query = llm2str.invoke([{"role": "user", "content": prompt}]).content
    rewrite_query_emb = embedding.embed_query(rewrite_query)
    query_emb = embedding.embed_query(query)
    from sklearn.metrics.pairwise import cosine_similarity
    import numpy as np
    print("原问题:"+query)
    print("改写后的问题:"+rewrite_query)
    embeddings = np.array([rewrite_query_emb, query_emb])
    similarity_matrix = cosine_similarity(embeddings)
    print(similarity_matrix[0, 1])
    return {
            "rewritten": rewritten,
            "rewrite_query": rewrite_query
        }


if __name__ == '__main__':
    messages=[
        {"role": "user", "content": "今天上海的天气怎么样"},
        {"role": "system", "content": "今天上海 18 度，晴天"},
        {"role": "user", "content": "明天呢？"},
        {"role": "system", "content": "明天预计 19 度，阴天"},
        {"role": "user", "content": "广州什么天气？"}
    ]
    print(digest_mrc(messages))


<|im_start|>system
你的任务是在多轮对话中，生成一个改写后的用户问题，以补全上下文中缺失的信息，一般的技巧是指代消岐和信息补全。<|im_end|>
<|im_start|>user
今天上海的天气怎么样
<|im_end|>
<|im_start|>assistant
今天上海 18 度，晴天
<|im_end|>
<|im_start|>user
明天呢？
<|im_end|>
<|im_start|>assistant
明天预计 19 度，阴天
<|im_end|>
-----请改写下面的用户问题，以补全上下文中缺失的信息，一般的技巧是指代消岐和信息补全-----
<|im_start|>user
广州什么天气？<|im_end|>
原问题:广州什么天气？
改写后的问题:广州今天的天气怎么样？
0.977381115909202
{'rewritten': False, 'rewrite_query': '广州今天的天气怎么样？'}


In [ ]:
# -*- coding: utf-8 -*-
# 第一次策略，没有判断是否需要重写。
from flask import Flask
from flask import request
from flask import jsonify
# coding=UTF-8
import json
import os
from flask import make_response
from openai import OpenAI
 
 
model_info = json.loads(os.getenv("LEADERBOARD_MODELHUB_KEY2INFO")) # 环境变量名称固定
token = model_info["token"] # 访问modelhub模型的token，固定key
base_url = model_info["entrypoint"] # 访问modelhub模型的base url，固定key
qwen72b_modelId = model_info["model_key2info"]["qwen72b"]["modelId"] # key的qwen72b与提交配置里的自定义名称对应
client = OpenAI(api_key=token, base_url=base_url)

# res = client.chat.completions.create(model=qwen72b_modelId, **request.get_json())

app = Flask(__name__)


# 在这里实现你的策略
def digest_mrc(messages):
    """
    # Request:
    {
    "messages": [
        {"role": "user", "content": "今天上海的天气怎么样"},
        {"role": "system", "content": "今天上海 18 度，晴天"},
        {"role": "user", "content": "明天呢？"},
        {"role": "system", "content": "明天预计 19 度，阴天"},
        {"role": "user", "content": "广州什么天气？"}
    ]
    }

    # Response:
    {
    "rewritten": true/false
    "rewrite_query": "广州明天什么天气？"
    }
    # 如果需要改写就是true，然后query就是改写后的问题，否则就是false，query字段可有可无
    """
    rewritten = False
    rewrite_query = messages[-1]["content"]
    # 策略 1 直接语言模型 提示改写
    ## 先将message处理成合适的prompt
    prompt_list = ["<|im_start|>system\n你的任务是根据多轮查询问题，将最新的问题改写成让用户只关注该问题就能理解这个问题<|im_end|>"]
    query = messages[-1]["content"]
    messages.pop(-1)
    for message in messages:
        if message["role"] == "user":
            prompt_list.append("<|im_start|>user\n" + message["content"] + "\n<|im_end|>")
        elif message["role"] == "system":
            prompt_list.append("<|im_start|>assistant\n" + message["content"] + "\n<|im_end|>")
        else:
            raise ValueError("Invalid role: " + message["role"])
    prompt_list.append("<|im_start|>user\n-----请改写下面这个问题，让用户只关注该问题就能理解这个问题-----\n" + query + "<|im_end|>")
    prompt = "\n".join(prompt_list)
    
    try:
        completion = client.chat.completions.create(model=qwen72b_modelId, messages=[{"role": "user", "content": prompt}])

        rewritten_query = completion.choices[0].message.content
        if rewritten_query != query:
            rewritten = True
            rewrite_query = rewritten_query
    except Exception as e:
        print(f"Error during OpenAI API call: {e}")
        return {
            "rewritten": rewritten,
            "rewrite_query": query
        }
    return {
        "rewritten": rewritten,
        "rewrite_query": rewrite_query
    }


# 此处不需要修改
@app.route("/predict",methods=["POST"])
def predict():
    data = request.get_json()
    messages = data.get('messages')

    return digest_mrc(messages)


# 此处不需要修改
if __name__ == '__main__':
   app.run("0.0.0.0",80)

In [ ]:
# -*- coding: utf-8 -*-
# 第二次策略，判断是否需要重写查询。向量相似度判断
from flask import Flask
from flask import request
from flask import jsonify
# coding=UTF-8
import json
import os
from flask import make_response
from openai import OpenAI
 
 
model_info = json.loads(os.getenv("LEADERBOARD_MODELHUB_KEY2INFO")) # 环境变量名称固定
token = model_info["token"] # 访问modelhub模型的token，固定key
base_url = model_info["entrypoint"] # 访问modelhub模型的base url，固定key
qwen72b_modelId = model_info["model_key2info"]["qwen72b"]["modelId"] # key的qwen72b与提交配置里的自定义名称对应
client = OpenAI(api_key=token, base_url=base_url)
embedding_model = model_info["model_key2info"]["embedding"]["modelId"]
# res = client.chat.completions.create(model=qwen72b_modelId, **request.get_json())

app = Flask(__name__)


# 在这里实现你的策略
def digest_mrc(messages):
    """
    # Request:
    {
    "messages": [
        {"role": "user", "content": "今天上海的天气怎么样"},
        {"role": "system", "content": "今天上海 18 度，晴天"},
        {"role": "user", "content": "明天呢？"},
        {"role": "system", "content": "明天预计 19 度，阴天"},
        {"role": "user", "content": "广州什么天气？"}
    ]
    }

    # Response:
    {
    "rewritten": true/false
    "rewrite_query": "广州明天什么天气？"
    }
    # 如果需要改写就是true，然后query就是改写后的问题，否则就是false，query字段可有可无
    """
    rewritten = False
    rewrite_query = messages[-1]["content"]
    # 策略 1 直接语言模型 提示改写
    ## 先将message处理成合适的prompt
    prompt_list = ["<|im_start|>system\n你的任务是根据多轮查询问题，将最新的问题改写成让用户只关注该问题就能理解这个问题<|im_end|>"]
    query = messages[-1]["content"]
    context = ""
    messages.pop(-1)
    for message in messages:
        context += message["content"] + "\n"
        if message["role"] == "user":
            prompt_list.append("<|im_start|>user\n" + message["content"] + "\n<|im_end|>")
        elif message["role"] == "system":
            prompt_list.append("<|im_start|>assistant\n" + message["content"] + "\n<|im_end|>")
        else:
            raise ValueError("Invalid role: " + message["role"])
    prompt_list.append("<|im_start|>user\n-----请改写下面这个问题，让用户只关注该问题就能理解这个问题-----\n" + query + "<|im_end|>")
    prompt = "\n".join(prompt_list)
    
    try:
        completion = client.chat.completions.create(model=qwen72b_modelId, messages=[{"role": "user", "content": prompt}])
        rewritten_query = completion.choices[0].message.content
        # from sentence_transformers import SentenceTransformer
        sentences_old = query
        sentences_new = rewritten_query
        # model = SentenceTransformer('lier007/xiaobu-embedding-v2')
        # embeddings_1 = model.encode(sentences_1, normalize_embeddings=True)
        # embeddings_2 = model.encode(sentences_2, normalize_embeddings=True)
        # similarity = embeddings_1 @ embeddings_2.T
        # print(similarity)

        context_embedding_response = client.embeddings.create(
            model=embedding_model,
            input=context,
        )
        context_embedding = context_embedding_response.data[0].embedding
        query_embedding_response_1 = client.embeddings.create(
            model=embedding_model,
            input=sentences_old,
        )
        query_embedding_1 = query_embedding_response_1.data[0].embedding
        query_embedding_response_2 = client.embeddings.create(
            model=embedding_model,
            input=sentences_new,
        )
        query_embedding_2 = query_embedding_response_2.data[0].embedding
        similarity = query_embedding_1 @ query_embedding_2.T # 原问题和新问题的相似性
        similarity_1 = query_embedding_1 @ context_embedding.T # 原问题和上下文的相似性
        similarity_2 = query_embedding_2 @ context_embedding.T # 新问题和上下文的相似性

        if similarity < 0.92 or similarity_2 > similarity_1: # 如果新问题与原问题相似度小于0.92或者新问题与上下文的相似度大于原问题与上下文的相似度，则改写
            rewritten = True
            rewrite_query = rewritten_query
    except Exception as e:
        print(f"Error during OpenAI API call: {e}")
        return {
            "rewritten": rewritten,
            "rewrite_query": query
        }
    return {
        "rewritten": rewritten,
        "rewrite_query": rewrite_query
    }


# 此处不需要修改
@app.route("/predict",methods=["POST"])
def predict():
    data = request.get_json()
    messages = data.get('messages')

    return digest_mrc(messages)


# 此处不需要修改
if __name__ == '__main__':
   app.run("0.0.0.0",80)

In [ ]:
# -*- coding: utf-8 -*-
# 第三次策略，判断是否需要重写查询。向量相似度判断+llm判断，改写类型有指代消岐和信息补全。主要在于如何判断是否需要改写。再考虑改写后的质量
# 0.0.6
from flask import Flask
from flask import request
from flask import jsonify
# coding=UTF-8
import json
import os
from flask import make_response
from openai import OpenAI
 
 
model_info = json.loads(os.getenv("LEADERBOARD_MODELHUB_KEY2INFO")) # 环境变量名称固定
token = model_info["token"] # 访问modelhub模型的token，固定key
base_url = model_info["entrypoint"] # 访问modelhub模型的base url，固定key
qwen72b_modelId = model_info["model_key2info"]["qwen72b"]["modelId"] # key的qwen72b与提交配置里的自定义名称对应
client = OpenAI(api_key=token, base_url=base_url)
embedding_model = model_info["model_key2info"]["embedding"]["modelId"]
# res = client.chat.completions.create(model=qwen72b_modelId, **request.get_json())

app = Flask(__name__)


# 在这里实现你的策略
def digest_mrc(messages):
    """
    # Request:
    {
    "messages": [
        {"role": "user", "content": "今天上海的天气怎么样"},
        {"role": "system", "content": "今天上海 18 度，晴天"},
        {"role": "user", "content": "明天呢？"},
        {"role": "system", "content": "明天预计 19 度，阴天"},
        {"role": "user", "content": "广州什么天气？"}
    ]
    }

    # Response:
    {
    "rewritten": true/false
    "rewrite_query": "广州明天什么天气？"
    }
    # 如果需要改写就是true，然后query就是改写后的问题，否则就是false，query字段可有可无
    """
    rewritten = False
    query = messages[-1]["content"]
    context = ""
    messages.pop(-1)
    for message in messages:
        context += message["content"] + "\n"
    
    try:
        # from sentence_transformers import SentenceTransformer
        sentences_old = query

        context_embedding_response = client.embeddings.create(
            model=embedding_model,
            input=context,
        )
        context_embedding = context_embedding_response.data[0].embedding
        query_embedding_response_1 = client.embeddings.create(
            model=embedding_model,
            input=sentences_old,
        )
        query_embedding_1 = query_embedding_response_1.data[0].embedding

        similarity_1 = query_embedding_1 @ context_embedding.T # 原问题和上下文的相似性

        if similarity_1 < 0.9:
            rewritten = True
            rewrite_query = query
    except Exception as e:
        print(f"Error during OpenAI API call: {e}")
        return {
            "rewritten": rewritten,
            "rewrite_query": query
        }
    return {
        "rewritten": rewritten,
        "rewrite_query": rewrite_query
    }


# 此处不需要修改
@app.route("/predict",methods=["POST"])
def predict():
    data = request.get_json()
    messages = data.get('messages')

    return digest_mrc(messages)


# 此处不需要修改
if __name__ == '__main__':
   app.run("0.0.0.0",80)

In [ ]:
# -*- coding: utf-8 -*-
# 0.0.7
# 第三次策略，判断是否需要重写查询。向量相似度判断+llm判断，改写类型有指代消岐和信息补全。主要在于如何判断是否需要改写。再考虑改写后的质量
from flask import Flask
from flask import request
from flask import jsonify
# coding=UTF-8
import json
import os
from flask import make_response
from openai import OpenAI
 
 
# res = client.chat.completions.create(model=qwen72b_modelId, **request.get_json())

app = Flask(__name__)


# 在这里实现你的策略
def digest_mrc(messages):
    """
    # Request:
    {
    "messages": [
        {"role": "user", "content": "今天上海的天气怎么样"},
        {"role": "system", "content": "今天上海 18 度，晴天"},
        {"role": "user", "content": "明天呢？"},
        {"role": "system", "content": "明天预计 19 度，阴天"},
        {"role": "user", "content": "广州什么天气？"}
    ]
    }

    # Response:
    {
    "rewritten": true/false
    "rewrite_query": "广州明天什么天气？"
    }
    # 如果需要改写就是true，然后query就是改写后的问题，否则就是false，query字段可有可无
    """
    rewritten = False
    rewrite_query = messages[-1]["content"]
    # 策略 1 直接语言模型 提示改写
    ## 先将message处理成合适的prompt
    query = messages[-1]["content"]
    context = ""
    messages.pop(-1)
    for message in messages:
        context += message["content"] + "\n"

    
    try:
        # from sentence_transformers import SentenceTransformer
        sentences_old = query
        from sentence_transformers import SentenceTransformer
        model = SentenceTransformer('moka-ai/m3e-base')
        embeddings_old = model.encode(sentences_old, normalize_embeddings=True)
        embeddings_context = model.encode(context, normalize_embeddings=True)
        similarity_old_with_context = embeddings_old @ embeddings_context.T # 原问题和上下文的相似性

        if similarity_old_with_context < 0.90:
            rewritten = True
            rewrite_query = query
    except Exception as e:
        print(f"Error during OpenAI API call: {e}")
        return {
            "rewritten": rewritten,
            "rewrite_query": query
        }
    return {
        "rewritten": rewritten,
        "rewrite_query": rewrite_query
    }


# 此处不需要修改
@app.route("/predict",methods=["POST"])
def predict():
    data = request.get_json()
    messages = data.get('messages')

    return digest_mrc(messages)


# 此处不需要修改
if __name__ == '__main__':
   app.run("0.0.0.0",80)

In [ ]:
# -*- coding: utf-8 -*-
# 第三次策略，判断是否需要重写查询。向量相似度判断+llm判断，改写类型有指代消岐和信息补全。主要在于如何判断是否需要改写。再考虑改写后的质量
# 0.0.6
from flask import Flask
from flask import request
from flask import jsonify
# coding=UTF-8
import json
import os
from flask import make_response
from openai import OpenAI
 
 
model_info = json.loads(os.getenv("LEADERBOARD_MODELHUB_KEY2INFO")) # 环境变量名称固定
token = model_info["token"] # 访问modelhub模型的token，固定key
base_url = model_info["entrypoint"] # 访问modelhub模型的base url，固定key
client = OpenAI(api_key=token, base_url=base_url)
embedding_model = model_info["model_key2info"]["embedding"]["modelId"]

app = Flask(__name__)


# 在这里实现你的策略
def digest_mrc(messages):
    """
    # Request:
    {
    "messages": [
        {"role": "user", "content": "今天上海的天气怎么样"},
        {"role": "system", "content": "今天上海 18 度，晴天"},
        {"role": "user", "content": "明天呢？"},
        {"role": "system", "content": "明天预计 19 度，阴天"},
        {"role": "user", "content": "广州什么天气？"}
    ]
    }

    # Response:
    {
    "rewritten": true/false
    "rewrite_query": "广州明天什么天气？"
    }
    # 如果需要改写就是true，然后query就是改写后的问题，否则就是false，query字段可有可无
    """
    rewritten = False
    query = messages[-1]["content"]
    context = ""
    messages.pop(-1)
    for message in messages:
        context += message["content"] + "\n"
    
    try:
        # from sentence_transformers import SentenceTransformer
        sentences_old = query

        context_embedding_response = client.embeddings.create(
            model=embedding_model,
            input=context,
        )
        context_embedding = context_embedding_response.data[0].embedding
        query_embedding_response_1 = client.embeddings.create(
            model=embedding_model,
            input=sentences_old,
        )
        query_embedding_1 = query_embedding_response_1.data[0].embedding

        similarity_1 = query_embedding_1 @ context_embedding.T # 原问题和上下文的相似性

        if similarity_1 < 0.9:
            rewritten = True
            rewrite_query = query
    except Exception as e:
        print(f"Error during OpenAI API call: {e}")
        return {
            "rewritten": rewritten,
            "rewrite_query": query
        }
    return {
        "rewritten": rewritten,
        "rewrite_query": rewrite_query
    }


# 此处不需要修改
@app.route("/predict",methods=["POST"])
def predict():
    data = request.get_json()
    messages = data.get('messages')

    return digest_mrc(messages)


# 此处不需要修改
if __name__ == '__main__':
   app.run("0.0.0.0",80)

In [9]:
import jieba
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def preprocess(text):
    return ' '.join(jieba.cut(text))

def calculate_cosine_similarity(question, context):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([question] + [context])
    cosine_similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:])
    return cosine_similarities[0][0]

question = "广州什么天气？"
context = "广州"

preprocessed_question = preprocess(question)
preprocessed_context = preprocess(context)

similarity = calculate_cosine_similarity(preprocessed_question, preprocessed_context)
print("相似度：", similarity)


相似度： 0.44943641652398214
